# LangChain Recipe and Shopping Recommender

In [ ]:
#!pip install selenium
#!pip install openai
#!pip install twilio
#!pip install langchain
#!pip install langchain_openai

In [ ]:
# Selenium for Browser automation and scraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# For sending get requests to web pages and apis
import requests

# Scraping tool
from bs4 import BeautifulSoup

# APIs that we will use
from openai import OpenAI
from twilio.rest import Client

# Langchain 
from langchain import PromptTemplate, LLMChain
from langchain_openai import ChatOpenAI
from langchain.chains import SimpleSequentialChain

In [ ]:
OPENAI_API_KEY="pass_your_own_api_key_as_a_string_here"
account_sid="pass_your_own_api_key_as_a_string_here"
auth_token="pass_your_own_auth_token_as_a_string_here"
my_twilio_number="your_twilio_number"
my_phone_number="your_own_phone_number"

In [ ]:
# Global Variables
url = "https://www.ica.se/erbjudanden/ica-supermarket-aptiten-1003988/"
client = OpenAI(api_key=OPENAI_API_KEY)
client_twilio = Client(account_sid, auth_token)


In [ ]:
def scrape_your_local_ica(url):    
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url)

    # Click Agree button
    agree_button_selector = "button#onetrust-accept-btn-handler"
    WebDriverWait(driver, 3)
    element = driver.find_element(By.CSS_SELECTOR, agree_button_selector)
    element.click()

    # scroll to bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # generate a beautiful soup
    soup = BeautifulSoup(driver.page_source)

    key = "article > div.offer-card__details-container"
    containers = soup.select(key)

    offers = [i.text.replace("\xa0kr. ", "") for i in containers]
    offers = [i.split(". ")[:2] for i in offers]

    driver.quit()
    return offers


def send_text(from_number, to_number, text_message):
    message = client_twilio.messages.create(
            body=text_message,
            from_=from_number,
            to=to_number)
    
def write_txt(file_name, content):
    with open(file_name, "w") as file:
        file.write(content)
        file.close()
        

In [ ]:
# Initialize LangChain pipeline
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.4, api_key=OPENAI_API_KEY) #"gpt-4o-mini"


# 1. Input Parsing Chain
input_parsing_template = """
You are a chef assistant. Based on the following weekly offers, generate a concise list of ingredients in JSON format:

Weekly offers:
{offers}

Respond with a JSON object.
"""
input_parsing_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(input_parsing_template))


# 2. Recipe Generation Chain
recipe_generation_template = """
Based on the following ingredients, recommend three simple dishes to cook and provide recipes for each. Ensure the dishes are easy to make:

Ingredients:
{ingredients}

Respond with the dish names and recipes.
"""
recipe_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(recipe_generation_template))


# 3. Shopping List Generation Chain
shopping_list_template = """
From the following recipes, create a consolidated shopping list with quantities:

Recipes:
{recipes}

Respond with the shopping list in bullet points.
"""
shopping_list_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(shopping_list_template))


# 4. Output Formatting Chain
output_formatting_template = """
Format the following shopping list for SMS delivery, keeping it concise:

Shopping list:
{shopping_list}

Respond with the formatted text.
"""
output_formatting_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(output_formatting_template))


# Scraping Local ICA supermarket
weekly_offers = scrape_your_local_ica(url)

# Creating a step-by-step LangChain to have full control of the individual steps
parsed_ingredients = input_parsing_chain.invoke({"offers":weekly_offers})
recipes = recipe_chain.invoke({"ingredients": parsed_ingredients["text"]})
shopping_list = shopping_list_chain.invoke({"recipes":recipes["text"]})
formatted_sms = output_formatting_chain.invoke({"shopping_list": shopping_list["text"]})

# Send shopping list as sms
send_text(my_twilo_number, my_phone_number, formatted_sms["shopping_list"])

# Write the recipes to text file
write_txt("Weekly_recipes.txt", shopping_list["recipes"])